<a href="https://colab.research.google.com/github/niyatikapoor/SVM-Multithreading/blob/main/102003732.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler
import os, sys, time, threading, multiprocessing, random as r

In [66]:
df= pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data", header= None)

In [67]:
df.columns= ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight','Rings']

In [68]:
df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [69]:
df.describe()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [70]:
dummies = pd.get_dummies(df.Sex, drop_first= False)
 
# Concatenate the dummies to original dataframe
df = pd.concat([dummies, df], axis='columns')
 
# drop the values
df.drop(['Sex','I'], axis='columns', inplace= True)
 
# print the dataframe
print(df)

      F  M  Length  Diameter  Height  Whole weight  Shucked weight  \
0     0  1   0.455     0.365   0.095        0.5140          0.2245   
1     0  1   0.350     0.265   0.090        0.2255          0.0995   
2     1  0   0.530     0.420   0.135        0.6770          0.2565   
3     0  1   0.440     0.365   0.125        0.5160          0.2155   
4     0  0   0.330     0.255   0.080        0.2050          0.0895   
...  .. ..     ...       ...     ...           ...             ...   
4172  1  0   0.565     0.450   0.165        0.8870          0.3700   
4173  0  1   0.590     0.440   0.135        0.9660          0.4390   
4174  0  1   0.600     0.475   0.205        1.1760          0.5255   
4175  1  0   0.625     0.485   0.150        1.0945          0.5310   
4176  0  1   0.710     0.555   0.195        1.9485          0.9455   

      Viscera weight  Shell weight  Rings  
0             0.1010        0.1500     15  
1             0.0485        0.0700      7  
2             0.1415       

In [71]:
scaler = StandardScaler()
numerical_cols = ['Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [72]:
samples = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.3)
    samples.append((X_train, X_test, y_train, y_test))

In [73]:
samples

[(      F  M    Length  Diameter    Height  Whole weight  Shucked weight  \
  3565  1  0  0.424788  0.676409  0.489780      0.135129        0.005103   
  3177  1  0 -0.158164  0.021352 -0.107991     -0.350258       -0.308050   
  2632  0  0 -0.907673 -0.936039 -0.944870     -0.946795       -0.823964   
  2877  0  0 -0.532919 -0.482538  0.011563     -0.778541       -0.893803   
  3792  1  0  1.091018  1.129911  0.848442      1.311887        1.208150   
  ...  .. ..       ...       ...       ...           ...             ...   
  2030  0  1  0.549706  0.575631  0.609334      0.583806        0.708007   
  608   0  1 -1.282428 -1.238373 -0.944870     -1.225179       -1.315095   
  213   0  1 -0.199803 -0.029037  0.011563     -0.434895       -0.533340   
  605   0  0 -0.824395 -0.633705 -0.347099     -0.823409       -0.898309   
  894   0  0 -2.365053 -2.346931 -1.781749     -1.554548       -1.479557   
  
        Viscera weight  Shell weight  
  3565        0.368668      0.231123  
  3177 

In [74]:
def fitnessfunction(X_train, X_test, y_train, y_test, kernel, c, gamma):
    model= svm.SVC( C= c, kernel= kernel, gamma=gamma)
    model.fit(X_train, y_train)
    print("fit")
    predicted = model.predict(X_test)
        
    # Model Evaluation Accuracy
    accuracy = round(np.mean(y_test == predicted) * 100, 2)
    return accuracy 
    

In [75]:
nsamples = 10
accuracies = []
params= []
iteration= 5

kernelList = ['linear', 'rbf', 'sigmoid']
df_sample= pd.DataFrame(columns=['sample#', 'best accuracy', 'c', 'gamma', 'kernel'])
result =  pd.DataFrame(columns=['samples', 'Thread1', 'Thread2', 'Thread3', 'Thread4'],index=range(2))


In [76]:
def task(sample,bestAccuracy,bestGamma,bestC,i):
    X_train, X_test, y_train, y_test = sample
    
    print("Iterating on sample ", i+1)
    for j in range(1, iteration+1):
        print("iteration ",j)
        kernel= np.random.choice(kernelList)
        C = np.random.uniform(0,10)
        gamma = np.random.uniform(0, 10)
        print(kernel , C, gamma)
        accuracy= fitnessfunction(X_train, X_test, y_train, y_test, kernel, C, gamma)
        if accuracy > bestAccuracy:
                bestAccuracy = accuracy
                bestC= C
                bestGamma= gamma
                bestKernel= kernel

                    
    accuracies.append(bestAccuracy)
    params.append([bestC, bestGamma, bestKernel])

    df_sample.loc[len(df_sample.index)] = [i+1, bestAccuracy, bestC, bestGamma, bestKernel]
    return None 

In [77]:
# divide dataset into training and testing sets with 10 different samples

acc1 = []
acc2 = []
num=[20,40]
for l in range(2):
    samples = []
    for i in range(num[l]):
      X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.3)
      samples.append((X_train, X_test, y_train, y_test))
    for k in range(1,5):
      print("\n#########################Using ",k, " threads#########################\n")
      startTime = time.time()

      numberOfThreads = k

      activeThreads = threading.activeCount()

      # Main Program - Run Multiple Threads
      
      print("Program Started....")
      for i in range(num[l]):
          bestC = 0 
          bestGamma = 0 
          bestAccuracy=0
            
          sample = samples[i]
          cmd = sample
          t = threading.Thread(target=task , args=(cmd,bestAccuracy,bestGamma,bestC,i,))
          t.start()

          print ("\nsample is ",i)
            
          while True:
            if threading.activeCount() - activeThreads + 1 <= numberOfThreads:
              break
            time.sleep(1)



      # Waiting to finish all Threads
      while True:
          if threading.activeCount() == activeThreads:
              break
          else:
              print ("    Thread still running (left %d)..."%(threading.activeCount() - activeThreads))
              time.sleep(1)
          
      print(" All Thread ends")

      print("Program Finished")
      print("Total Time %f sec" % (round( time.time() - startTime,4)))
          #print(f"Sample {i+1}: Best accuracy = {bestAccuracy}, Best params = {bestNu, bestEpsilon, bestKernel}")
      print("Done iterating!\n")
      val = df_sample['best accuracy'].idxmax()
      df2= df_sample.loc[val]['best accuracy']
      if(l==0):
        acc1.append(df2)
      else:
        acc2.append(df2)


#########################Using  1  threads#########################

Program Started....
Iterating on sample  1
iteration  1
sigmoid 3.4851799017528027 6.829182880951995

sample is  0
fit
iteration  2
rbf 6.529390123377386 4.459626802650337
fit
iteration  3
linear 5.315961325840477 8.589495781724878
fit
iteration  4
rbf 3.295854985144259 8.202161928996958
fit
iteration  5
rbf 4.736554987686014 4.235581319591365
fit
Iterating on sample  2
iteration  1
linear 
sample is 2.217921374029869 1
 1.1917484621577212
fit
iteration  2
rbf 3.0874529057382913 2.14585607611859
fit
iteration  3
linear 6.764566780108843 0.7216734396131752
fit
iteration  4
rbf 3.3106394508963533 2.81674947736384
fit
iteration  5
rbf 5.104989872202825 3.64791863092061
fit
Iterating on sample  3
iteration  1
sigmoid 0.14360460113632123 6.102643318914125

sample is  2
fit
iteration  2
linear 3.0952148739525907 6.114052073236519
fit
iteration  3
linear 6.176654892727127 1.931574975499497
fit
iteration  4
linear 5.17707706

In [78]:
result.loc[0].samples = 20
result.loc[0].Thread1 = acc1[0]
result.loc[0].Thread2 = acc1[1]
result.loc[0].Thread3 = acc1[2]
result.loc[0].Thread4 = acc1[3]
result.loc[1].samples = 40
result.loc[1].Thread1 = acc2[0]
result.loc[1].Thread2 = acc2[1]
result.loc[1].Thread3 = acc2[2]
result.loc[1].Thread4 = acc2[3]
print(result)

  samples Thread1 Thread2 Thread3 Thread4
0      20   27.19   27.19   27.27   27.35
1      40   27.67   27.67   27.67   27.83
